# SWIFTCON 2025 - The "fiducial" simulation pipeline

This notebook takes you through an example of running a dark-matter-only simulation and creating subhalo catalogues for subsequent analysis. The software used throughout this tutorial is the following:

* **Initial conditions**: monofonIC. [Code repository](https://bitbucket.org/ohahn/monofonic/src/master/) and latest [reference paper](https://ui.adsabs.harvard.edu/abs/2020ascl.soft08024H/abstract).
* **Cosmological integration**: SWIFT. [Code repository](https://gitlab.cosma.dur.ac.uk/swift/swiftsim) and [reference paper](https://ui.adsabs.harvard.edu/abs/2024MNRAS.530.2378S/abstract).
* **Subhalo finding**: HBT-HERONS. [Code repository](https://github.com/SWIFTSIM/HBT-HERONS) and [reference paper](https://ui.adsabs.harvard.edu/abs/2025MNRAS.543.1339F/abstract).
* **Subhalo property calculation**: SOAP. [Code repository](https://github.com/SWIFTSIM/SOAP) and [reference paper](https://ui.adsabs.harvard.edu/abs/2025JOSS...10.8252M/abstract).

Created by Victor Forouhar Moreno (forouhar@strw.leidenuniv.nl) & Rob McGibbon (mcgibbon@strw.leidenuniv.nl)

## Dependencies

Several external libraries are required to compile and/or run the codes used in this tutorial, although there is considerable overlap across codes. If you are using the provided Docker image, all libraries are pre-installed and ready to go. If you are not using the provided image, you will need to install these libraries yourself.

#### monofonIC
- FFTW
- GSL
- HDF5

#### SWIFT
- HDF5 
- MPI
- FFTW
- METIS
- GSL

#### HBT-HERONS
- CMake
- HDF5
- MPI

#### SOAP
- mpi4py
- h5py built with parallel HDF5
- Standard python modules (see requirements.txt)

#### Setup on cosma

This cell sets up dependencies for those running on cosma

In [ ]:
module purge
module load python/3.12.4 gnu_comp/14.1.0 openmpi/5.0.3 parallel_hdf5/1.12.3
module load fftw gsl parmetis
venv_name="$(echo $HOME | sed 's/home/apps/')/swift_workshop_env"
python="$venv_name/bin/python3"

# Generating initial conditions

We need to download monofonIC first, for which we clone the official repository.

In [ ]:
git clone https://bitbucket.org/ohahn/monofonic.git ./software/monofonic

We then compile the code to generate the executable we will be using.

In [ ]:
cd ./software/monofonic/
mkdir build/ && cd build

# Compilation options can be optionally specified at this stage.
cmake ..
make -j 4

# Go back to original directory
cd ../../..

### Running monofonIC

The monofonIC executable takes the path to a parameter file as a runtime argument. In this parameter file, you can specify the redshift at which the ICs will be generated, the cosmological parameters and , among other things. 

We have provided a basic parameter file in `./parameter_files/monofonic/example.conf`. Note that the following mandatory parameters have been left unspecified, as we encourage you to play with their value:

* `GridRes`: number of particles per dimension. Total number of particles will be the cube of this number.
* `BoxLength`: Length of each side of the cubic box. **It should be in `Mpc/h`!** 



In [ ]:
/home/jupyteruser/tutorial/software/monofonic/build/monofonIC ./parameter_files/monofonic/example.conf

# SWIFT - Running simulation

[SWIFT](https://swift.strw.leidenuniv.nl/docs/index.html) is an open-source cosmological and astrophysical numerical solver designed to run efficiently on modern hardware. A comprehensive and extensive set of models for galaxy formation as well as planetary physics are provided alongside a large series of examples.

#### Clone repository

In [ ]:
git clone https://gitlab.cosma.dur.ac.uk/swift/swiftsim.git
cd swiftsim

#### Configure and compile

See the [notes here](https://swift.strw.leidenuniv.nl/docs/GettingStarted/compiling_code.html)

Since HBTplus requires FOF information, we need to pass the `--enable-fof` flag

Hydro considerations
- Lots

In [ ]:
./autogen.sh
./configure --enable-fof

In [ ]:
make -j 4
cd ..

#### Set parameters

Define the units used by SWIFT, the cosmology of the run, and some parameters for running the simulation.

Specify output location and frequency. Make sure to enable running the FOF before writing a snapshot.

Information about how to run the FOF.

Point the code to the initial conditions we generated.

In [ ]:
cat > small_cosmo_volume_dm.yml << EOF
# Define the system of units to use internally. 
InternalUnitSystem:
  UnitMass_in_cgs:     1.98841e43    # 10^10 M_sun
  UnitLength_in_cgs:   3.08567758e24 # 1 Mpc
  UnitVelocity_in_cgs: 1e5           # 1 km/s
  UnitCurrent_in_cgs:  1             # Amperes
  UnitTemp_in_cgs:     1             # Kelvin

Cosmology:
  Omega_cdm:      0.3145
  Omega_b:        0.0
  Omega_lambda:   0.6855 
  h:              0.673
  a_begin:        0.02      	     # z_ini = 50.
  a_end:          1.0		         # z_end = 0.

# Parameters governing the time integration
TimeIntegration:
  dt_min:     1e-6 
  dt_max:     1e-2 

# Parameters for the self-gravity scheme
Gravity:
  eta:          0.025
  MAC:          adaptive
  theta_cr:     0.7
  epsilon_fmm:  0.001
  comoving_DM_softening:     0.0889     # 1/25th of the mean inter-particle separation: 88.9 kpc
  max_physical_DM_softening: 0.0889     # 1/25th of the mean inter-particle separation: 88.9 kpc
  mesh_side_length:       64

# Parameters governing the snapshots
Snapshots:
  basename:            snap
  delta_time:          1.15
  scale_factor_first:  0.1     # z = 9
  compression:         4
  invoke_fof:          1

# Parameters for running the friend-of-friends algorithm
FOF:  
  basename: fof_output                         # Filename for the FOF outputs.
  min_group_size: 20                           # The minimum no. of particles required for a group.
  linking_length_ratio: 0.2                    # Linking length in units of the mean inter-particle separation
  seed_black_holes_enabled: 0
  linking_types: [0, 1, 0, 0, 0, 0, 0]         # Use DM as the primary FOF linking type
  attaching_types: [0, 0, 0, 0, 0, 0, 0]

# Parameters governing the conserved quantities statistics
Statistics:
  delta_time:          1.01
  scale_factor_first:  0.03

# Parameters governing the cell tree
Scheduler:
  max_top_level_cells: 8
  cell_split_size:     50
  
# Parameters related to the initial conditions
InitialConditions:
  file_name:                   small_cosmo_volume.hdf5
  periodic:                    1
EOF

#### Run the code

Here we are running the non-MPI version of the code

In [ ]:
./swiftsim/swift --cosmology --self-gravity --fof --threads=4 small_cosmo_volume_dm.yml

#### Output of run

- `snap_xxxx.hdf5` - Particles and their properties
- `fof_output_xxxx.hdf5` - FOF halo catalogue (FOF IDs stored in snapshots)
- `statistics.txt` - Global properties of the simulation over time
- `timesteps.txt` - What the simulation did during it's timesteps

In [ ]:
ls

# HBTplus - Halo finding

HBT uses the FOF catalogues from SWIFT to as candidates for central subhalos. Once a subhalo has been identified at any snapshot, the particles remain associated to it even if it falls into a larger halo. Using this information it is possible to identify satellite subhalos.

This method requires that HBTplus is run on a range of snapshots from the simulation, it cannot be run on a single snapshot. We recommend at least ~64 snapshot spaced evenly in $\log a$ (we did less in this notebook since it's just an example).

Hydro considerations
- SWIFT particle splitting
- Which particles to use as tracers
- Number of tracers for an object to be resolved
- Thermal energy of gas particles

#### Clone

In [ ]:
git clone https://github.com/SWIFTSIM/HBTplus.git
cd HBTplus

#### Configure and compile

Use `ccmake` to see what options are available

In [ ]:
cmake -B$PWD/build -D HBT_USE_OPENMP=ON -D HBT_DM_ONLY=ON -D HBT_UNSIGNED_LONG_ID_OUTPUT=OFF

In [ ]:
cd build
make -j 4
cd ../..

#### Set parameters

In [ ]:
cat > HBT_config.txt << EOF
# Configuration file used to run on small DMO simulation 

# Compulsary Params
SnapshotPath ./                      # Location of snapshots
SnapshotFileBase snap                # Basename of snapshot files
HaloPath ./                          # Location of FOF ID files (normal snapshot for SWIFT)
SubhaloPath HBT_output               # Output directory
ParticlesSplit 0                     # Do we have gas particles that split
MergeTrappedSubhalos 1               # Allow subhalos to merge
# For the swiftsim reader these will be read in from snapshots automatically
BoxSize -1
SofteningHalo -1
MaxPhysicalSofteningHalo -1

# Reader
SnapshotFormat swiftsim
GroupFileFormat swiftsim_particle_index
MinSnapshotIndex 0                  # First snapshot to use
MaxSnapshotIndex 18                 # Final snapshot to use
MaxConcurrentIO 8                   # Number of cores for IO
MinNumPartOfSub 20                  # Minimum number of particles in a subhalo

# Units
MassInMsunh 6.81e9                  # Removes h factors from the final output 
LengthInMpch 0.681                  # Removes h factors from the final output
VelInKmS 1
EOF

##### Run

Run with a single rank as we will use 8 threads

For FLAMINGO we were often IO limited, but this is not the case for COLIBRE

In [ ]:
export OMP_NUM_THREADS=8
mpirun -np 1 ./HBTplus/build/HBT HBT_config.txt

#### Output of run

For each snapshot each HBT rank outputs two files.
- The `SubSnap_xxx.y.hdf5` files contain information about the bound subhalos identified by HBTplus, including their particles. There are only a very small number of halo properties contained in these files. All information for merger trees is also contained within these files.
- The `SrcSnap_xxx.y.hdf5` files contain a list of the particles associated with each subhalo, but which are not bound at the current snapshot. These files are only used if HBT if restarted, and so can be deleted once halo finding has been completed.

HBT contains "orphan" subhalos. These are subhalos which have been disrupted, but are still tracked by the most bound particle at the time the last snapshot the subhalo was resolved.

In [ ]:
echo "Contents of HBT_output:"
ls HBT_output
echo "Output for snapshot 18:"
ls HBT_output/018

# SOAP - Calculating halo properties

Dependencies
- mpi4py
- h5py built with parallel HDF5
- Standard python modules (see requirements.txt)

#### Clone repo

In [ ]:
git clone https://github.com/SWIFTSIM/SOAP.git
cd SOAP
pip install -e .

#### Parameter file

Set input and output directories



In [ ]:
cat > SOAP_config.yml << EOF
#### Parameter file

# Values in this section are substituted into the other sections
# The simulation name (box size and resolution) and snapshot will be appended
# to these to get the full name of the input/output files/directories
Parameters:
  sim_dir: $(pwd)
  output_dir: $(pwd)
  scratch_dir: $(pwd)

# Location of the Swift snapshots:
Snapshots:
  # Use {snap_nr:04d} for the snapshot number and {file_nr} for the file number.
  filename: "{sim_dir}/snap_{snap_nr:04d}.hdf5"

# Which halo finder we're using, and base name for halo finder output files
HaloFinder:
  type: HBTplus
  filename: "{sim_dir}/HBT_output/{snap_nr:03d}/SubSnap_{snap_nr:03d}"
  fof_filename: "{sim_dir}/fof_output_{snap_nr:04d}.hdf5"

GroupMembership:
  # Where to write the group membership files
  filename: "{output_dir}/SOAP_uncompressed/membership_{snap_nr:04d}.hdf5"

HaloProperties:
  # Where to write the halo properties file
  filename: "{output_dir}/SOAP_uncompressed/halo_properties_{snap_nr:04d}.hdf5"
  # Where to write temporary chunk output
  chunk_dir: "{scratch_dir}/SOAP-tmp/{halo_finder}/"

ApertureProperties:
  properties:
    {}
  variations:
    {}
ProjectedApertureProperties:
  properties:
    {}
  variations:
    {}
SOProperties:
  properties:
    CentreOfMass: true
    CentreOfMassVelocity: true
    Concentration: true
    MassFractionSatellites: true
    MassFractionExternal: true
    NumberOfDarkMatterParticles: true
    SORadius: true
    SpinParameter: true
    TotalMass: true
  variations:
    200_crit:
      type: crit
      value: 200.0
    200_mean:
      type: mean
      value: 200.0
    500_crit:
      type: crit
      value: 500.0
SubhaloProperties:
  properties:
    CentreOfMass: true
    CentreOfMassVelocity: true
    NumberOfDarkMatterParticles: true
    MaximumCircularVelocity: true
    MaximumCircularVelocityUnsoftened: true
    MaximumCircularVelocityRadiusUnsoftened: true
    SpinParameter: true
    TotalMass: true
  variations:
    Bound:
      bound_only: true
filters:
  general:
    limit: 100
    properties:
      - BoundSubhalo/NumberOfGasParticles
      - BoundSubhalo/NumberOfDarkMatterParticles
      - BoundSubhalo/NumberOfStarParticles
      - BoundSubhalo/NumberOfBlackHoleParticles
    combine_properties: sum
  baryon:
    limit: 100
    properties:
      - BoundSubhalo/NumberOfGasParticles
      - BoundSubhalo/NumberOfStarParticles
    combine_properties: sum
  dm:
    limit: 100
    properties:
      - BoundSubhalo/NumberOfDarkMatterParticles
  gas:
    limit: 100
    properties:
      - BoundSubhalo/NumberOfGasParticles
  star:
    limit: 100
    properties:
      - BoundSubhalo/NumberOfStarParticles
calculations:
  recalculate_xrays: false
  calculate_missing_properties: false
  min_read_radius_cmpc: 5
  recently_heated_gas_filter:
    delta_time_myr: 15
    use_AGN_delta_T: true
EOF

#### Running the group membership files

Should be relatively cheap, just reads in the particles for each halo, resorts them, and outputs them in a SWIFT friendly format

In [ ]:
mpirun -np 4 python3 -u SOAP/SOAP/group_membership.py \
    --sim-name=DM_test \
    --snap-nr=18 \
    SOAP_config.yml

#### Calculating halo properties

Chunks sets how to split up the simulation volume. We require multiple chunks if running on multiple nodes.

In [ ]:
mpirun -np 4 python3 -u SOAP/SOAP/compute_halo_properties.py \
    --sim-name=DM_test \
    --snap-nr=18 \
    --chunks=1 \
    --dmo \
    SOAP_config.yml

#### SOAP output

Membership files have the same structure as a SWIFT snapshot. There are no particle IDs, but the particles are in the same order as the original snapshot.

SOAP catalogues have a group for each halo type. All arrays have the same length (the number of subhalos), and are always in the same order.

See Rob's workshop on Friday

In [ ]:
ls SOAP_uncompressed

#### Compression

The files output from SOAP can be heavily compressed. For the membership files this is because most particles are not in a halo, and there are many repeated indices for the ones that are. For the halo properties we do not compute certain properties depending on the size of the input halo, so many values are zero.

The membership files have no lossy compression filters, and can be compressed with h5repack

In [ ]:
h5repack -i SOAP_uncompressed/membership_0018.hdf5 -o membership_0018.hdf5 -f GZIP=4

The halo catalogues have the same lossy compression filters as are available in SWIFT (some of which are custom), and so must be compressed using the following script.

In [ ]:
python SOAP/compression/compress_fast_metadata.py \
    SOAP_uncompressed/halo_properties_0018.hdf5 \
    halo_properties_0018.hdf5 \
    SOAP_uncompressed/tmp

#### Generate documentation

Documentation can be generated by running the `property_table.py`. You must pass the parameter file (to determine which halo types and properties are included in the documentation) and a SWIFT snapshot (to extract the units).

In [ ]:
cd SOAP
$python property_table.py ../SOAP_config.yml ../snap_0018.hdf5
cd documentation
# pdflatex SOAP.tex
cd ../..